In [1]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import skimage.io as io
import skimage.filters as flt
from skimage.metrics import structural_similarity as SSIM
%matplotlib inline
# since we can't use imports
import numpy as np
import scipy.ndimage.filters as flt
import warnings
import cv2
from glob import glob 

import pylab as pl
from time import sleep
import tensorflow as tf
from tqdm import tqdm
from collections import Counter
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib as mpl


import sys
sys.path.append('../../scripts')
import open_frame
from pm_algorithm import anisodiff
from data_augmentation import get_generators
from architectures import get_model


CROP = 256
image_size = (CROP,CROP)
typ = "gaussian2"
open_frame = getattr(open_frame,typ)


import logging
tf.get_logger().setLevel(logging.ERROR)

2023-01-26 02:20:58.374589: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/joel/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-26 02:20:58.374606: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
gen_batch_train,gen_batch_val = get_generators(typ,var1_d=1,var1_u=7,CROP1=CROP,BATCH_SIZE=10)

2023-01-26 02:20:59.813287: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/joel/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-01-26 02:20:59.813312: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-26 02:20:59.813332: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (joel-Latitude-5410): /proc/driver/nvidia/version does not exist
2023-01-26 02:20:59.813602: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
xs, ys = next(gen_batch_val.as_numpy_iterator())

In [116]:
partition_low = tf.constant(np.power(np.linspace(0,1,num_classes+1),2)[:-1])
partition_low = tf.expand_dims(tf.expand_dims(tf.expand_dims(partition_low,0),0),0)
partition_low = tf.cast(partition_low,tf.float32)
partition_up = tf.constant(np.power(np.linspace(0,1,num_classes+1),2)[1:])
partition_up = tf.expand_dims(tf.expand_dims(tf.expand_dims(partition_up,0),0),0)
partition_up = tf.cast(partition_up,tf.float32)


dx_n = tf.keras.layers.Lambda(lambda z: z/tf.reduce_max(tf.math.abs(z)))(dx)


ineq1_x = tf.greater_equal(dx_n, partition_low)
ineq2_x = tf.less(dx_n,partition_up)

interval_x = tf.cast(tf.math.logical_and(ineq1_x,ineq2_x),tf.float32)

In [126]:
order = 1
num_classes = 10
it_lim = 10
input_shape = image_size + (1,)
lamb = 50
gamma = 0.1


inputs = tf.keras.Input(shape=input_shape,name='input')
input_emb = tf.keras.layers.Input(shape = (1),name='input_emb')
emb = tf.keras.layers.Dense(num_classes*(order+1),activation='relu')(input_emb)
emb = tf.keras.layers.Dense(num_classes*(order+1),activation='relu')(emb)
emb = tf.keras.layers.Reshape((num_classes,order+1),name='embedding')(emb)
x = tf.keras.layers.Conv2D(CROP//2, kernel_size=3, activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(3,3))(x)
x = tf.keras.layers.Conv2D(CROP//4, kernel_size=3, activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3,3))(x)
x = tf.keras.layers.Conv2D(CROP//8, kernel_size=3, activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3,3))(x)

y = tf.keras.layers.Flatten()(x)

partition_low = tf.constant(np.power(np.linspace(0,1,num_classes+1),2)[:-1])
partition_low = tf.expand_dims(tf.expand_dims(tf.expand_dims(partition_low,0),0),0)
partition_low = tf.cast(partition_low,tf.float32)
partition_up = tf.constant(np.power(np.linspace(0,1,num_classes+1),2)[1:])
partition_up = tf.expand_dims(tf.expand_dims(tf.expand_dims(partition_up,0),0),0)
partition_up = tf.cast(partition_up,tf.float32)

outputs = inputs

for num_it in range(it_lim):

    dx = tf.keras.layers.Lambda(lambda z: z[:,:-2,1:-1] - z[:,1:-1,1:-1])(outputs)
    dy = tf.keras.layers.Lambda(lambda z: z[:,2:,1:-1] - z[:,1:-1,1:-1])(outputs)
    dz = tf.keras.layers.Lambda(lambda z: z[:,1:-1,2:] - z[:,1:-1,1:-1])(outputs)
    dw = tf.keras.layers.Lambda(lambda z: z[:,1:-1,:-2] - z[:,1:-1,1:-1])(outputs)
    
    dx_n = tf.keras.layers.Lambda(lambda z: z/tf.reduce_max(tf.math.abs(z)))(dx)
    dy_n = tf.keras.layers.Lambda(lambda z: z/tf.reduce_max(tf.math.abs(z)))(dy)
    dz_n = tf.keras.layers.Lambda(lambda z: z/tf.reduce_max(tf.math.abs(z)))(dz)
    dw_n = tf.keras.layers.Lambda(lambda z: z/tf.reduce_max(tf.math.abs(z)))(dw)
    
    dx2 = tf.keras.layers.Lambda(lambda z:tf.math.pow(z,2))(dx_n)
    dy2 = tf.keras.layers.Lambda(lambda z:tf.math.pow(z,2))(dy_n)
    dz2 = tf.keras.layers.Lambda(lambda z:tf.math.pow(z,2))(dz_n)
    dw2 = tf.keras.layers.Lambda(lambda z:tf.math.pow(z,2))(dw_n)


    ineq1_x = tf.greater_equal(dx_n, partition_low)
    ineq2_x = tf.less(dx_n,partition_up)
    ineq1_y = tf.greater_equal(dy_n, partition_low)
    ineq2_y = tf.less(dy_n,partition_up)
    ineq1_z = tf.greater_equal(dz_n, partition_low)
    ineq2_z = tf.less(dz_n,partition_up)
    ineq1_w = tf.greater_equal(dw_n, partition_low)
    ineq2_w = tf.less(dw_n,partition_up)

    interval_x = tf.cast(tf.math.logical_and(ineq1_x,ineq2_x),tf.float32)
    interval_y = tf.cast(tf.math.logical_and(ineq1_y,ineq2_y),tf.float32)
    interval_z = tf.cast(tf.math.logical_and(ineq1_z,ineq2_z),tf.float32)
    interval_w = tf.cast(tf.math.logical_and(ineq1_w,ineq2_w),tf.float32)


    if num_it == 0:

        ct = tf.keras.layers.Dense(num_classes*(order+1),activation='linear')(y)
        ct = tf.keras.layers.Reshape((num_classes,order+1),name=f'coeff_spline_{num_it}')(ct)
        ct = tf.keras.layers.multiply([ct,emb])
        ct = tf.keras.layers.Lambda(lambda z: tf.expand_dims(tf.expand_dims(z,axis=1),axis=1))(ct)

    power_norm_x = tf.pow(dx2,tf.constant(np.asarray(np.arange(order+1),dtype='float32')))
    power_norm_x = tf.keras.layers.Lambda(lambda z: tf.expand_dims(z,axis=-2))(power_norm_x)
    power_norm_y = tf.pow(dy2,tf.constant(np.asarray(np.arange(order+1),dtype='float32')))
    power_norm_y = tf.keras.layers.Lambda(lambda z: tf.expand_dims(z,axis=-2))(power_norm_y)
    power_norm_z = tf.pow(dz2,tf.constant(np.asarray(np.arange(order+1),dtype='float32')))
    power_norm_z = tf.keras.layers.Lambda(lambda z: tf.expand_dims(z,axis=-2))(power_norm_z)
    power_norm_w = tf.pow(dw2,tf.constant(np.asarray(np.arange(order+1),dtype='float32')))
    power_norm_w = tf.keras.layers.Lambda(lambda z: tf.expand_dims(z,axis=-2))(power_norm_w)
    

    spline_x = tf.keras.layers.multiply([ct,power_norm_x])
    spline_x = tf.keras.layers.Lambda(lambda z: tf.unstack(z,axis=-1))(spline_x)
    spline_x = tf.keras.layers.add(spline_x)
    spline_x = tf.keras.layers.multiply([spline_x,interval_x])
    spline_y = tf.keras.layers.multiply([ct,power_norm_y])
    spline_y = tf.keras.layers.Lambda(lambda z: tf.unstack(z,axis=-1))(spline_y)
    spline_y = tf.keras.layers.add(spline_y)
    spline_y = tf.keras.layers.multiply([spline_y,interval_y])
    spline_z = tf.keras.layers.multiply([ct,power_norm_z])
    spline_z = tf.keras.layers.Lambda(lambda z: tf.unstack(z,axis=-1))(spline_z)
    spline_z = tf.keras.layers.add(spline_z)
    spline_z = tf.keras.layers.multiply([spline_z,interval_z])
    spline_w = tf.keras.layers.multiply([ct,power_norm_w])
    spline_w = tf.keras.layers.Lambda(lambda z: tf.unstack(z,axis=-1))(spline_w)
    spline_w = tf.keras.layers.add(spline_w)
    spline_w = tf.keras.layers.multiply([spline_w,interval_w])

    coeff_x = tf.expand_dims(tf.keras.layers.add(tf.unstack(spline_x,axis=-1),name=f'coeff_x_{num_it}'),axis=-1)
    coeff_y = tf.expand_dims(tf.keras.layers.add(tf.unstack(spline_y,axis=-1),name=f'coeff_y_{num_it}'),axis=-1)
    coeff_z = tf.expand_dims(tf.keras.layers.add(tf.unstack(spline_z,axis=-1),name=f'coeff_z_{num_it}'),axis=-1)
    coeff_w = tf.expand_dims(tf.keras.layers.add(tf.unstack(spline_w,axis=-1),name=f'coeff_w_{num_it}'),axis=-1)


    outputs_x = tf.keras.layers.multiply([coeff_x,dx])
    outputs_y = tf.keras.layers.multiply([coeff_y,dy])
    outputs_z = tf.keras.layers.multiply([coeff_z,dz])
    outputs_w = tf.keras.layers.multiply([coeff_w,dw])

    outputs_it = tf.keras.layers.add([outputs_x,outputs_y,outputs_z,outputs_w])
    new_outputs = tf.ones_like(outputs_it)
    zeros_y = tf.expand_dims(tf.zeros_like(outputs_it)[:,1],axis=-1)
    zeros_x = tf.expand_dims(tf.zeros_like(outputs)[:,1],axis=-3)
    pad_y = tf.keras.layers.Concatenate(axis=-2)([zeros_y,outputs_it,zeros_y])
    new_pad_y = tf.keras.layers.Concatenate(axis=-2)([zeros_y,new_outputs,zeros_y])
    outputs_it = tf.keras.layers.Concatenate(axis=1)([zeros_x,pad_y,zeros_x])
    new_outputs = tf.keras.layers.Concatenate(axis=1)([zeros_x,new_pad_y,zeros_x])
    new_outputs = tf.keras.layers.multiply([new_outputs,outputs])
    new_outputs = tf.keras.layers.add([0.1*outputs_it,new_outputs])
    new_outputs = tf.keras.layers.Lambda(lambda z: tf.clip_by_value(z,0,1))(new_outputs)
    outputs = new_outputs


In [180]:
order = 1
num_classes = 3
it_lim = 2
input_shape = image_size + (1,)
lamb = 50
gamma = 0.1

inputs = xs['input']
input_emb = xs['input_emb']

emb = tf.keras.layers.Dense(num_classes*(order+1),activation='relu')(input_emb)
emb = tf.keras.layers.Dense(num_classes*(order+1),activation='relu')(emb)
emb = tf.keras.layers.Reshape((num_classes,order+1),name='embedding')(emb)
x = tf.keras.layers.Conv2D(CROP//2, kernel_size=3, activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D(pool_size=(3,3))(x)
x = tf.keras.layers.Conv2D(CROP//4, kernel_size=3, activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3,3))(x)
x = tf.keras.layers.Conv2D(CROP//8, kernel_size=3, activation='relu')(x)
x = tf.keras.layers.MaxPool2D(pool_size=(3,3))(x)

y = tf.keras.layers.Flatten()(x)

partition_low = tf.constant(np.power(np.linspace(0,1,num_classes+1),2)[:-1])
partition_low = tf.expand_dims(tf.expand_dims(tf.expand_dims(partition_low,0),0),0)
partition_low = tf.cast(partition_low,tf.float32)
partition_up = tf.constant(np.power(np.linspace(0,1,num_classes+1),2)[1:])
partition_up = tf.expand_dims(tf.expand_dims(tf.expand_dims(partition_up,0),0),0)
partition_up = tf.cast(partition_up,tf.float32)

outputs = inputs

for num_it in range(it_lim):

    dx = tf.keras.layers.Lambda(lambda z: z[:,:-2,1:-1] - z[:,1:-1,1:-1])(outputs)
    dy = tf.keras.layers.Lambda(lambda z: z[:,2:,1:-1] - z[:,1:-1,1:-1])(outputs)
    dz = tf.keras.layers.Lambda(lambda z: z[:,1:-1,2:] - z[:,1:-1,1:-1])(outputs)
    dw = tf.keras.layers.Lambda(lambda z: z[:,1:-1,:-2] - z[:,1:-1,1:-1])(outputs)
    
    dx_n = tf.keras.layers.Lambda(lambda z: tf.math.abs(z)/tf.reduce_max(tf.math.abs(z)))(dx)
    dy_n = tf.keras.layers.Lambda(lambda z: tf.math.abs(z)/tf.reduce_max(tf.math.abs(z)))(dy)
    dz_n = tf.keras.layers.Lambda(lambda z: tf.math.abs(z)/tf.reduce_max(tf.math.abs(z)))(dz)
    dw_n = tf.keras.layers.Lambda(lambda z: tf.math.abs(z)/tf.reduce_max(tf.math.abs(z)))(dw)
    
    dx2 = tf.keras.layers.Lambda(lambda z:tf.math.pow(z,2))(dx_n)
    dy2 = tf.keras.layers.Lambda(lambda z:tf.math.pow(z,2))(dy_n)
    dz2 = tf.keras.layers.Lambda(lambda z:tf.math.pow(z,2))(dz_n)
    dw2 = tf.keras.layers.Lambda(lambda z:tf.math.pow(z,2))(dw_n)


    ineq1_x = tf.greater_equal(dx_n, partition_low)
    ineq2_x = tf.less_equal(dx_n,partition_up)
    ineq1_y = tf.greater_equal(dy_n, partition_low)
    ineq2_y = tf.less_equal(dy_n,partition_up)
    ineq1_z = tf.greater_equal(dz_n, partition_low)
    ineq2_z = tf.less_equal(dz_n,partition_up)
    ineq1_w = tf.greater_equal(dw_n, partition_low)
    ineq2_w = tf.less_equal(dw_n,partition_up)

    interval_x = tf.cast(tf.math.logical_and(ineq1_x,ineq2_x),tf.float32)
    interval_y = tf.cast(tf.math.logical_and(ineq1_y,ineq2_y),tf.float32)
    interval_z = tf.cast(tf.math.logical_and(ineq1_z,ineq2_z),tf.float32)
    interval_w = tf.cast(tf.math.logical_and(ineq1_w,ineq2_w),tf.float32)


    if num_it == 0:

        ct = tf.keras.layers.Dense(num_classes*(order+1),activation='linear')(y)
        ct = tf.keras.layers.Reshape((num_classes,order+1),name=f'coeff_spline_{num_it}')(ct)
        ct = tf.keras.layers.multiply([ct,emb])
        ct = tf.keras.layers.Lambda(lambda z: tf.expand_dims(tf.expand_dims(z,axis=1),axis=1))(ct)

    power_norm_x = tf.pow(dx2,tf.constant(np.asarray(np.arange(order+1),dtype='float32')))
    power_norm_x = tf.keras.layers.Lambda(lambda z: tf.expand_dims(z,axis=-2))(power_norm_x)
    power_norm_y = tf.pow(dy2,tf.constant(np.asarray(np.arange(order+1),dtype='float32')))
    power_norm_y = tf.keras.layers.Lambda(lambda z: tf.expand_dims(z,axis=-2))(power_norm_y)
    power_norm_z = tf.pow(dz2,tf.constant(np.asarray(np.arange(order+1),dtype='float32')))
    power_norm_z = tf.keras.layers.Lambda(lambda z: tf.expand_dims(z,axis=-2))(power_norm_z)
    power_norm_w = tf.pow(dw2,tf.constant(np.asarray(np.arange(order+1),dtype='float32')))
    power_norm_w = tf.keras.layers.Lambda(lambda z: tf.expand_dims(z,axis=-2))(power_norm_w)
    

    spline_x = tf.keras.layers.multiply([ct,power_norm_x])
    spline_x = tf.keras.layers.Lambda(lambda z: tf.unstack(z,axis=-1))(spline_x)
    spline_x = tf.keras.layers.add(spline_x)
    spline_x = tf.keras.layers.multiply([spline_x,interval_x])
    spline_y = tf.keras.layers.multiply([ct,power_norm_y])
    spline_y = tf.keras.layers.Lambda(lambda z: tf.unstack(z,axis=-1))(spline_y)
    spline_y = tf.keras.layers.add(spline_y)
    spline_y = tf.keras.layers.multiply([spline_y,interval_y])
    spline_z = tf.keras.layers.multiply([ct,power_norm_z])
    spline_z = tf.keras.layers.Lambda(lambda z: tf.unstack(z,axis=-1))(spline_z)
    spline_z = tf.keras.layers.add(spline_z)
    spline_z = tf.keras.layers.multiply([spline_z,interval_z])
    spline_w = tf.keras.layers.multiply([ct,power_norm_w])
    spline_w = tf.keras.layers.Lambda(lambda z: tf.unstack(z,axis=-1))(spline_w)
    spline_w = tf.keras.layers.add(spline_w)
    spline_w = tf.keras.layers.multiply([spline_w,interval_w])

    coeff_x = tf.expand_dims(tf.keras.layers.add(tf.unstack(spline_x,axis=-1),name=f'coeff_x_{num_it}'),axis=-1)
    coeff_y = tf.expand_dims(tf.keras.layers.add(tf.unstack(spline_y,axis=-1),name=f'coeff_y_{num_it}'),axis=-1)
    coeff_z = tf.expand_dims(tf.keras.layers.add(tf.unstack(spline_z,axis=-1),name=f'coeff_z_{num_it}'),axis=-1)
    coeff_w = tf.expand_dims(tf.keras.layers.add(tf.unstack(spline_w,axis=-1),name=f'coeff_w_{num_it}'),axis=-1)


    outputs_x = tf.keras.layers.multiply([coeff_x,dx])
    outputs_y = tf.keras.layers.multiply([coeff_y,dy])
    outputs_z = tf.keras.layers.multiply([coeff_z,dz])
    outputs_w = tf.keras.layers.multiply([coeff_w,dw])

    outputs_it = tf.keras.layers.add([outputs_x,outputs_y,outputs_z,outputs_w])
    new_outputs = tf.ones_like(outputs_it)
    zeros_y = tf.expand_dims(tf.zeros_like(outputs_it)[:,1],axis=-1)
    zeros_x = tf.expand_dims(tf.zeros_like(outputs)[:,1],axis=-3)
    pad_y = tf.keras.layers.Concatenate(axis=-2)([zeros_y,outputs_it,zeros_y])
    new_pad_y = tf.keras.layers.Concatenate(axis=-2)([zeros_y,new_outputs,zeros_y])
    outputs_it = tf.keras.layers.Concatenate(axis=1)([zeros_x,pad_y,zeros_x])
    new_outputs = tf.keras.layers.Concatenate(axis=1)([zeros_x,new_pad_y,zeros_x])
    new_outputs = tf.keras.layers.multiply([new_outputs,outputs])
    new_outputs = tf.keras.layers.add([0.1*outputs_it,new_outputs])
    new_outputs = tf.keras.layers.Lambda(lambda z: tf.clip_by_value(z,0,1))(new_outputs)
    outputs = new_outputs

In [18]:
def f(lamb,b):
    return tf.math.exp(-1* (np.power(lamb,2))/(np.power(b,2)))

In [77]:
it_lim = 10
lambd = lamb

i = np.random.choice(len(xs['input']))


img = np.copy(xs['input'][i])

img_new = np.zeros(img.shape)

for t in range(it_lim): 
    dx_ = img[:-2,1:-1] - img[1:-1,1:-1] 
    dy_ = img[2:,1:-1] - img[1:-1,1:-1] 
    dz_ = img[1:-1,2:] - img[1:-1,1:-1] 
    dw_ = img[1:-1,:-2] - img[1:-1,1:-1] 


    img_new[1:-1,1:-1] = img[1:-1,1:-1] +gamma * (f(dx_,lambd)*dx_ + f (dy_,lambd)*dy_ + f (dz_,lambd)*dz_ + f (dw_,lambd)*dw_) 
    img = img_new 

    


0.0005266182